In [20]:
from io import StringIO
import base64

import nltk
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
from nltk.corpus import stopwords
import ssl

from nltk.stem import PorterStemmer
from nltk.stem.snowball import SnowballStemmer

nltk.download('wordnet')

from nltk import sent_tokenize
from nltk import word_tokenize

from nltk.stem import WordNetLemmatizer 

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

import torch
from transformers import BertTokenizer

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [21]:
df=pd.read_csv(r'C:\Users\hp\Downloads\gpt3.5_output8.csv')
df

,Unnamed: 0,message_id,subject,content,Body,result
0,0,<6184043.1075857585289.JavaMail.evans@thyme>,more about you,A little bird told me about this. Read it wh...,more about you A little bird told me about thi...,"4) Logistic Arrangements (meeting scheduling, ..."
1,1,<17858209.1075857584947.JavaMail.evans@thyme>,LNG on the road,trucking LNG to California - The short ans...,LNG on the road trucking LNG to California - ...,"4) 'Logistic Arrangements(meeting scheduling, ..."
2,2,<14187877.1075857584924.JavaMail.evans@thyme>,Amazon.com Password Assistance,Greetings from Amazon.com. To finish resetti...,Amazon.com Password Assistance Greetings from ...,2) 'Purely Personal'
3,3,<31194989.1075857585233.JavaMail.evans@thyme>,Deferral Enrollment 2001,...,Deferral Enrollment 2001 ...,"4) 'Logistic Arrangements(meeting scheduling, ..."
4,4,<32084772.1075857584968.JavaMail.evans@thyme>,LNG Questions,Russ: A couple of questions - Check the DOE ...,LNG Questions Russ: A couple of questions - C...,"4) 'Logistic Arrangements(meeting scheduling, ..."
...,...,...,...,...,...,...
8658,10936,<5783114.1075842092300.JavaMail.evans@thyme>,Re: Puget Sound,Joe: I think that Tana is right. I guess tha...,Re: Puget Sound Joe: I think that Tana is righ...,"4) 'Logistic Arrangements(meeting scheduling, ..."
8659,10937,<10827699.1075842088710.JavaMail.evans@thyme>,Pancanadian Energy Services,Russell: Since late we have been trying to p...,Pancanadian Energy Services Russell: Since lat...,"4) 'Logistic Arrangements(meeting scheduling, ..."
8660,10938,<679593.1075842088856.JavaMail.evans@thyme>,Project Margaux - Revised Swap Documentation,-----------------,Project Margaux - Revised Swap Documentation -...,6) 'Document editing/checking(collaboration)'
8661,10939,<5961861.1075842092626.JavaMail.evans@thyme>,ISDA Securities Rep,Greg: Can you help me answer this question? ...,ISDA Securities Rep Greg: Can you help me answ...,"1) 'Company Business, Strategy, etc.'"


In [22]:
df.drop(['Unnamed: 0','Body'],axis=1,inplace=True)
df

,message_id,subject,content,result
0,<6184043.1075857585289.JavaMail.evans@thyme>,more about you,A little bird told me about this. Read it wh...,"4) Logistic Arrangements (meeting scheduling, ..."
1,<17858209.1075857584947.JavaMail.evans@thyme>,LNG on the road,trucking LNG to California - The short ans...,"4) 'Logistic Arrangements(meeting scheduling, ..."
2,<14187877.1075857584924.JavaMail.evans@thyme>,Amazon.com Password Assistance,Greetings from Amazon.com. To finish resetti...,2) 'Purely Personal'
3,<31194989.1075857585233.JavaMail.evans@thyme>,Deferral Enrollment 2001,...,"4) 'Logistic Arrangements(meeting scheduling, ..."
4,<32084772.1075857584968.JavaMail.evans@thyme>,LNG Questions,Russ: A couple of questions - Check the DOE ...,"4) 'Logistic Arrangements(meeting scheduling, ..."
...,...,...,...,...
8658,<5783114.1075842092300.JavaMail.evans@thyme>,Re: Puget Sound,Joe: I think that Tana is right. I guess tha...,"4) 'Logistic Arrangements(meeting scheduling, ..."
8659,<10827699.1075842088710.JavaMail.evans@thyme>,Pancanadian Energy Services,Russell: Since late we have been trying to p...,"4) 'Logistic Arrangements(meeting scheduling, ..."
8660,<679593.1075842088856.JavaMail.evans@thyme>,Project Margaux - Revised Swap Documentation,-----------------,6) 'Document editing/checking(collaboration)'
8661,<5961861.1075842092626.JavaMail.evans@thyme>,ISDA Securities Rep,Greg: Can you help me answer this question? ...,"1) 'Company Business, Strategy, etc.'"


In [38]:
df

,message_id,subject,content,result
0,<6184043.1075857585289.JavaMail.evans@thyme>,more about you,A little bird told me about this. Read it wh...,4
1,<17858209.1075857584947.JavaMail.evans@thyme>,LNG on the road,trucking LNG to California - The short ans...,4
2,<14187877.1075857584924.JavaMail.evans@thyme>,Amazon.com Password Assistance,Greetings from Amazon.com. To finish resetti...,2
3,<31194989.1075857585233.JavaMail.evans@thyme>,Deferral Enrollment 2001,...,4
4,<32084772.1075857584968.JavaMail.evans@thyme>,LNG Questions,Russ: A couple of questions - Check the DOE ...,4
...,...,...,...,...
8658,<5783114.1075842092300.JavaMail.evans@thyme>,Re: Puget Sound,Joe: I think that Tana is right. I guess tha...,4
8659,<10827699.1075842088710.JavaMail.evans@thyme>,Pancanadian Energy Services,Russell: Since late we have been trying to p...,4
8660,<679593.1075842088856.JavaMail.evans@thyme>,Project Margaux - Revised Swap Documentation,-----------------,6
8661,<5961861.1075842092626.JavaMail.evans@thyme>,ISDA Securities Rep,Greg: Can you help me answer this question? ...,1


In [39]:
'''
def create_onedrive_directdownload (onedrive_link):
    data_bytes64 = base64.b64encode(bytes(onedrive_link, 'utf-8'))
    data_bytes64_String = data_bytes64.decode('utf-8').replace('/','_').replace('+','-').rstrip("=")
    resultUrl = f"https://api.onedrive.com/v1.0/shares/u!{data_bytes64_String}/root/content"
    return resultUrl

onedrive_link=create_onedrive_directdownload("https://1drv.ms/x/s!AoiE7xOoBAsngsd-PpntYUKRt9nOQg?e=YO5X5h")
print(onedrive_link)
'''

'\ndef create_onedrive_directdownload (onedrive_link):\n    data_bytes64 = base64.b64encode(bytes(onedrive_link, \'utf-8\'))\n    data_bytes64_String = data_bytes64.decode(\'utf-8\').replace(\'/\',\'_\').replace(\'+\',\'-\').rstrip("=")\n    resultUrl = f"https://api.onedrive.com/v1.0/shares/u!{data_bytes64_String}/root/content"\n    return resultUrl\n\nonedrive_link=create_onedrive_directdownload("https://1drv.ms/x/s!AoiE7xOoBAsngsd-PpntYUKRt9nOQg?e=YO5X5h")\nprint(onedrive_link)\n'

In [40]:
'''
df=pd.read_excel(onedrive_link)
df.head()
'''

'\ndf=pd.read_excel(onedrive_link)\ndf.head()\n'

In [42]:
'''
# RE for filtering raw mail
df['message_id']=df['RawMailContent'].str.extract('message_id'+r':\s(.*)\n')
df['subject']=df['RawMailContent'].str.extract('Subject'+r':\s(.*)\n')
df['content']=df['RawMailContent'].str.extract('X-FileName'+r':\s.*\n([\s\S]*)$')
df.drop('RawMailContent',axis=1,inplace=True)
df['content']=df['content'].replace({'\n':' ','\t':' '},regex=True)

df['content']=df['content'].replace({'"':'',"'":""},regex=True)
df['subject']=df['subject'].replace({'"':'',"'":""},regex=True)

df.loc[df['content'].str.contains('-----Original Message-----'),'content']
df.loc[df['content'].str.contains('-----Original Message-----'),'content']
df.head()
'''

'\n# RE for filtering raw mail\ndf[\'message_id\']=df[\'RawMailContent\'].str.extract(\'message_id\'+r\':\\s(.*)\n\')\ndf[\'subject\']=df[\'RawMailContent\'].str.extract(\'Subject\'+r\':\\s(.*)\n\')\ndf[\'content\']=df[\'RawMailContent\'].str.extract(\'X-FileName\'+r\':\\s.*\n([\\s\\S]*)$\')\ndf.drop(\'RawMailContent\',axis=1,inplace=True)\ndf[\'content\']=df[\'content\'].replace({\'\n\':\' \',\'\t\':\' \'},regex=True)\n\ndf[\'content\']=df[\'content\'].replace({\'"\':\'\',"\'":""},regex=True)\ndf[\'subject\']=df[\'subject\'].replace({\'"\':\'\',"\'":""},regex=True)\n\ndf.loc[df[\'content\'].str.contains(\'-----Original Message-----\'),\'content\']\ndf.loc[df[\'content\'].str.contains(\'-----Original Message-----\'),\'content\']\ndf.head()\n'

In [43]:
# df=df[~df['content'].isna()]
# df=df[df['labeled']==True]
# col=['message_id','subject','content','cat_1_level_2']
# df=df.loc[:,col]
df

,message_id,subject,content,result
0,<6184043.1075857585289.JavaMail.evans@thyme>,more about you,A little bird told me about this. Read it wh...,4
1,<17858209.1075857584947.JavaMail.evans@thyme>,LNG on the road,trucking LNG to California - The short ans...,4
2,<14187877.1075857584924.JavaMail.evans@thyme>,Amazon.com Password Assistance,Greetings from Amazon.com. To finish resetti...,2
3,<31194989.1075857585233.JavaMail.evans@thyme>,Deferral Enrollment 2001,...,4
4,<32084772.1075857584968.JavaMail.evans@thyme>,LNG Questions,Russ: A couple of questions - Check the DOE ...,4
...,...,...,...,...
8658,<5783114.1075842092300.JavaMail.evans@thyme>,Re: Puget Sound,Joe: I think that Tana is right. I guess tha...,4
8659,<10827699.1075842088710.JavaMail.evans@thyme>,Pancanadian Energy Services,Russell: Since late we have been trying to p...,4
8660,<679593.1075842088856.JavaMail.evans@thyme>,Project Margaux - Revised Swap Documentation,-----------------,6
8661,<5961861.1075842092626.JavaMail.evans@thyme>,ISDA Securities Rep,Greg: Can you help me answer this question? ...,1


In [44]:
df['content']=df['content'].replace({'\n':' ','\t':' '},regex=True)
df['content']=df['content'].replace({',':'',},regex=True)
df['content']=df['content'].replace({';':'',},regex=True)

df['content']=df['content'].replace({'"':'',},regex=True)
df['subject']=df['subject'].replace({'"':''},regex=True)

df['content']=df['content'].replace({"'":""},regex=True)
df['subject']=df['subject'].replace({"'":""},regex=True)

df=df.loc[(~df['subject'].str.startswith('FW:',na=False))  ,:]
df.loc[(~df['subject'].str.startswith('FW:',na=False))  ,:]

df.loc[df['content'].str.contains('-----Original Message-----'),'content']=df.loc[df['content'].str.contains('-----Original Message-----'),'content'].str.extract(r'([\s\S]*?)'+r'-----Original Message-----',expand=False)
df.loc[df['content'].str.contains('----- Forwarded by'),'content']=df.loc[df['content'].str.contains('----- Forwarded by'),'content'].str.extract(r'([\s\S]*?)'+r'----- Forwarded by',expand=False)
df.loc[df['content'].str.contains('---------------------- Forwarded by'),'content']=df.loc[df['content'].str.contains('---------------------- Forwarded by'),'content'].str.extract(r'([\s\S]*?)'+r'---------------------- Forwarded by',expand=False)
df.loc[df['content'].str.contains('-------------------------- Sent from my BlackBerry'),'content']=df.loc[df['content'].str.contains('-------------------------- Sent from my BlackBerry'),'content'].str.extract(r'([\s\S]*?)'+r'-------------------------- Sent from my BlackBerry',expand=False)

# remove all the numbers
df['content']=df['content'].replace({r'\d+':''},regex=True)

# remove email ids
df['content']=df['content'].replace({r'\S*@\S*\s?':''},regex=True)

'''
# remove everything after To: From: Re: Subject: etc..
df['content']=df['content'].replace({r'To:.+':'',r'From:.+':'',r'Re:.+':'',r'Subject:.+':'',r'Date:.+':''},regex=True)
'''

# remove just To: From: Re: Subject: etc.. texts
df['content']=df['content'].replace({r'To:':'',r'From:':'',r'Re:':'',r'Subject:':'',r'Date:':''},regex=True)




df

C:\Users\hp\AppData\Local\Temp\ipykernel_4796\3058441772.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['content']=df['content'].replace({'\n':' ','\t':' '},regex=True)
C:\Users\hp\AppData\Local\Temp\ipykernel_4796\3058441772.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['content']=df['content'].replace({',':'',},regex=True)
C:\Users\hp\AppData\Local\Temp\ipykernel_4796\3058441772.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .lo

,message_id,subject,content,result
0,<6184043.1075857585289.JavaMail.evans@thyme>,more about you,A little bird told me about this. Read it wh...,4
1,<17858209.1075857584947.JavaMail.evans@thyme>,LNG on the road,trucking LNG to California - The short ans...,4
2,<14187877.1075857584924.JavaMail.evans@thyme>,Amazon.com Password Assistance,Greetings from Amazon.com. To finish resetti...,2
3,<31194989.1075857585233.JavaMail.evans@thyme>,Deferral Enrollment 2001,...,4
4,<32084772.1075857584968.JavaMail.evans@thyme>,LNG Questions,Russ: A couple of questions - Check the DOE ...,4
...,...,...,...,...
8658,<5783114.1075842092300.JavaMail.evans@thyme>,Re: Puget Sound,Joe: I think that Tana is right. I guess tha...,4
8659,<10827699.1075842088710.JavaMail.evans@thyme>,Pancanadian Energy Services,Russell: Since late we have been trying to p...,4
8660,<679593.1075842088856.JavaMail.evans@thyme>,Project Margaux - Revised Swap Documentation,-----------------,6
8661,<5961861.1075842092626.JavaMail.evans@thyme>,ISDA Securities Rep,Greg: Can you help me answer this question? ...,1


In [46]:
'''
# removing duplicates
# df.rename(columns={'Message-ID': 'message_id'}, inplace=True)
df1=df1['Message-ID']
df1=list(df1)
df=df[~df['message_id'].isin(df1)]
df
'''

"\n# removing duplicates\n# df.rename(columns={'Message-ID': 'message_id'}, inplace=True)\ndf1=df1['Message-ID']\ndf1=list(df1)\ndf=df[~df['message_id'].isin(df1)]\ndf\n"

In [46]:
df['message_id']=df['message_id'].astype(str)
df['subject']=df['subject'].astype(str)
df['content']=df['content'].astype(str)

# to lower type
df['content']=df['content'].str.lower()
df['subject']=df['subject'].str.lower()

# remove punctuations
df['content'] = df['content'].str.replace('[^\w\s]','')
df['subject'] = df['subject'].str.replace('[^\w\s]','')

'''
# remove combination of words and digits
df['content'] = df['content'].str.replace('W*dw*','')
df['subject'] = df['subject'].str.replace('W*dw*','')
'''

# tokenization
df["content"] = df["content"].apply(lambda x: nltk.word_tokenize(x))
df["subject"] = df["subject"].apply(lambda x: nltk.word_tokenize(x))

'''
# porter stemming
ps = PorterStemmer()
df["content"] = df["content"].apply(lambda x: [ps.stem(i) for i in x])
df["subject"] = df["subject"].apply(lambda x: [ps.stem(i) for i in x])
'''
'''

# snowball stemming
ss = SnowballStemmer(language = "english")
df["content"] = df["content"].apply(lambda x: [ss.stem(i) for i in x])
df["subject"] = df["subject"].apply(lambda x: [ss.stem(i) for i in x])
'''

# lemmantization
lemmatizer = WordNetLemmatizer()
df["content"] = df["content"].apply(lambda x: [lemmatizer.lemmatize(i) for i in x])
df["subject"] = df["subject"].apply(lambda x: [lemmatizer.lemmatize(i) for i in x])

# join text back
def join_words(text):
    return " ".join(text)
df["subject"] = df["subject"].apply(lambda x: join_words(x))
df["content"] = df["content"].apply(lambda x: join_words(x))

df.to_csv('gpt3.5_output8.csv')

C:\Users\hp\AppData\Local\Temp\ipykernel_4796\9208195.py:10: FutureWarning: The default value of regex will change from True to False in a future version.
  df['content'] = df['content'].str.replace('[^\w\s]','')
C:\Users\hp\AppData\Local\Temp\ipykernel_4796\9208195.py:11: FutureWarning: The default value of regex will change from True to False in a future version.
  df['subject'] = df['subject'].str.replace('[^\w\s]','')


In [47]:
df

,message_id,subject,content,result
0,<6184043.1075857585289.JavaMail.evans@thyme>,more about you,a little bird told me about this read it when ...,4
1,<17858209.1075857584947.JavaMail.evans@thyme>,lng on the road,trucking lng to california the short answer is...,4
2,<14187877.1075857584924.JavaMail.evans@thyme>,amazoncom password assistance,greeting from amazoncom to finish resetting yo...,2
3,<31194989.1075857585233.JavaMail.evans@thyme>,deferral enrollment 2001,the annual deferral program enrollment process...,4
4,<32084772.1075857584968.JavaMail.evans@thyme>,lng question,rus a couple of question check the doe northea...,4
...,...,...,...,...
8658,<5783114.1075842092300.JavaMail.evans@thyme>,re puget sound,joe i think that tana is right i guess that if...,4
8659,<10827699.1075842088710.JavaMail.evans@thyme>,pancanadian energy service,russell since late we have been trying to put ...,4
8660,<679593.1075842088856.JavaMail.evans@thyme>,project margaux revised swap documentation,,6
8661,<5961861.1075842092626.JavaMail.evans@thyme>,isda security rep,greg can you help me answer this question also...,1


In [ ]:
df_content=df['content']
df_message_id=df['message_id']
df_subject=df['subject']
df_label=df['cat_1_level_2'].astype('int')

In [ ]:
df.to_csv('BERT_data.csv', index=False)

In [ ]:
# character length
df['content'].str.len().hist()

In [ ]:
# total word length
def sentence_len(sentence):
    return len(sentence.split())

df['content'].apply(lambda x: sentence_len(x)).hist()

In [ ]:
# average word length
df['content'].str.split().apply(lambda x : [len(i) for i in x]).map(lambda x: np.mean(x)).hist()

1.	CC	Coordinating conjunction
2.	CD	Cardinal number
3.	DT	Determiner
4.	EX	Existential there
5.	FW	Foreign word
6.	IN	Preposition or subordinating conjunction
7.	JJ	Adjective
8.	JJR	Adjective, comparative
9.	JJS	Adjective, superlative
10.	LS	List item marker
11.	MD	Modal
12.	NN	Noun, singular or mass
13.	NNS	Noun, plural
14.	NNP	Proper noun, singular
15.	NNPS	Proper noun, plural
16.	PDT	Predeterminer
17.	POS	Possessive ending
18.	PRP	Personal pronoun
19.	PRP$	Possessive pronoun
20.	RB	Adverb
21.	RBR	Adverb, comparative
22.	RBS	Adverb, superlative
23.	RP	Particle
24.	SYM	Symbol
25.	TO	to
26.	UH	Interjection
27.	VB	Verb, base form
28.	VBD	Verb, past tense
29.	VBG	Verb, gerund or present participle
30.	VBN	Verb, past participle
31.	VBP	Verb, non-3rd person singular present
32.	VBZ	Verb, 3rd person singular present
33.	WDT	Wh-determiner
34.	WP	Wh-pronoun
35.	WP$$	Possessive wh-pronoun
36.	WRB	Wh-adverb

In [ ]:
# word count
new= df['content'].str.split()
new=df['content'].values.tolist()
corpus=[word for i in new for word in i.split()]
wo_li={}
for wo in corpus: 
    wo_li[wo] = wo_li.get(wo, 0) + 1    

In [ ]:
wo_li = dict(sorted(wo_li.items(), key=lambda item: item[1], reverse=True))
wo_li

pos_tags = pos_tag(wo_li.keys())
pos_tags

In [ ]:
pos_li={}
for x,pos in pos_tags: 
    pos_li[pos] = pos_li.get(pos, 0) + 1 
pos_li = dict(sorted(pos_li.items(), key=lambda item: item[1], reverse=True))
plt.figure(figsize=(10, 6))
sns.barplot(x=list(pos_li.keys())[:15],y=list(pos_li.values())[:15],palette='viridis')
plt.title('POS Distribution',size=22)
plt.xlabel('Part of Speech')
plt.ylabel('Count')

In [ ]:
pos_tags[1][1]

In [ ]:
nltk.download('stopwords')
stop=set(stopwords.words('english'))

In [ ]:
# stop words
stop_wo_li={}
for wo,i in wo_li.items():
    if wo in stop:
        stop_wo_li[wo]=i

plt.figure(figsize=(10, 6))
sns.barplot(x=list(stop_wo_li.keys())[:15],y=list(stop_wo_li.values())[:15],palette='viridis')
plt.title('Stop Words Distribution',size=22)
plt.xlabel('Stop Word')
plt.ylabel('Count')

In [ ]:
# not stop words
for st_wo in stop:
    if st_wo in wo_li:
        wo_li.pop(st_wo)
wo_li
plt.bar(list(wo_li.keys())[:5],list(wo_li.values())[:5])

In [ ]:
'''
# filtering out stop words
stop_words = set(stopwords.words('english'))
def remove_stop_words(text):
    tokens = nltk.word_tokenize(text)
    filtered_tokens = [word for word in tokens if word.lower() not in stop_words]
    return ' '.join(filtered_tokens)
df['content'] = df['content'].apply(remove_stop_words)
'''

In [ ]:
df["content"]

In [1]:
'''
# for removing names
import spacy
from spacy import displacy
from collections import Counter
nlp = spacy.load("en_core_web_sm")
import en_core_web_sm
nlp = en_core_web_sm.load()
doc = nlp('jim authorities fined Google a record $5.1 billion on Wednesday for abusing its power in the mobile phone market and ordered the company to alter its practices')
print([(X.text, X.label_) for X in doc.ents])
'''

In [ ]:
df.to_csv('BERT_data.csv', index=False)